#Import libraries

### download packages

In [ ]:
!pip install razdel

!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install 'DAWG-Python >= 0.7'

!pip install slovnet

!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_morph_news_v1.tar
!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar

!pip3 install deeppavlov
!pip install transformers
!python -m deeppavlov install bert_sentence_embedder

#download pre-trained BERT model and unzip it
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
!tar xvzf rubert_cased_L-12_H-768_A-12_pt.tar.gz

! wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ru.vec

     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 8.2MB 12.2MB/s 
     |████████████████████████████████| 7.1MB 3.8MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
--2021-05-24 15:18:09--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  36.5MB/s    in 0.7s    

2021-05-24 15:18:10 (36.5 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]

--2021-05-24 15:18:11--  https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_morph_news_v1.tar
Resolving storage.yandexcloud.net (storage.yandexc

     |████████████████████████████████| 2.3MB 13.0MB/s 
     |████████████████████████████████| 3.3MB 54.5MB/s 
2021-05-24 15:19:01.700 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'bert_sentence_embedder' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/embedder/bert_sentence_embedder.json'
     |████████████████████████████████| 573kB 13.9MB/s 
     |████████████████████████████████| 1.2MB 28.7MB/s 
     |████████████████████████████████| 5.6MB 29.7MB/s 
     |████████████████████████████████| 133kB 57.3MB/s 
     |████████████████████████████████| 7.6MB 44.4MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
ERROR: botocore 1.20.78 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2
  Found existing installation: transformers 4.6.1
    Uninstalling transformers-4.6

In [ ]:
assert 1==0,'Restart rumtime in previous cell!'

AssertionError: ignored

### import packages

In [ ]:
#Base
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:

#NLP
import re

import pymorphy2

from string import punctuation
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph,NER

from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs

#Metric
from gensim.models import *



### Set models

In [ ]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)
print('Done!')

Done!


In [ ]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)
print('Done!')

Done!


In [ ]:
#set model
bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'rubert_cased_L-12_H-768_A-12_pt/'

bert = build_model(bert_config)
print('Done!')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


Done!


In [ ]:
model = KeyedVectors.load_word2vec_format('wiki.ru.vec', binary=False)

# Load data

In [ ]:
#Add google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#open file
with open("/content/drive/MyDrive/Diplom/Generation/s2t/text.txt", encoding= 'cp1251') as txt:
  text = txt.read()
text

'"Семь навыков высокоэффективных людей" - это одна из самых популярных книг для развития личности, которая была переведена на 38 языков. Многомиллионные тиражи говорят о том, что среди литературы по саморазвитию она обладает непререкаемым авторитетом. Эта книга более 7 лет возглавляла список бестселлеров и по результатам опроса читателей журнала для руководителей была названа самой влиятельной книгой по бизнесу 20 века. Журнал Time назвал автора книги Доктора Стивена Кови одним из 25 самых влиятельных американцев. В данном видео вы узнаете ключевые тезисы этой уникальной книги. В начале книги Стивен Кови объясняет понятие навыка как пересечение знания, умения и желания. Знание это теоретическая база, определяющая, что делать и зачем. Умение определяет как делать, а желание - это мотивация, то есть хочу делать. Чтобы что-то в своей жизни превратить в навык, необходимы все три компонента.\n"Семь навыков" - это навыки эффективности. Базируясь на принципах, они приносят наилучшие из возмож

In [ ]:
df_bleu = pd.read_csv('https://raw.githubusercontent.com/Digital-Pushkin-Lab/RuAdapt/main/df_bleu.csv', sep= ',', error_bad_lines=False)
df_bleu.head()

,source,target,level
0,"— И очень хорошо сделали, что не обратились к ...","— И очень хорошо сделали, что не обратились к ...",b1_c1
1,"Чужестранец все прохаживался, но, увидев, что ...","Батюшки мои, думаю себе: вот чудак-то! И прика...",b1_c1
2,"Шипучин. Нет, я не вынесу! (Плачет.) Не вынесу...","Шипучин. Нет, я не вынесу! (Плачет.) Не вынесу...",a2_b1
3,"— Очень много, — краснея и с глупой, небрежной...","— Очень много, — краснея и с глупой улыбкой, к...",b1_c1
4,Вечер и вправду складывался весело.,"Вечер и, правда, становился всё более весёлым....",b1


### Load RSL dict with onyms

In [ ]:
path = "/content/drive/MyDrive/Diplom/Generation/s2t/"

In [ ]:
RSL_dict = pd.read_csv(path + 'RSL_ONYMS.csv', index_col= 0 )
RSL_dict['RSL WORD ID'] = RSL_dict.index
RSL_dict.head()

,WORD,PART_OF_SPEECH,TOPIC_CATEGORY,MAIN_CATEGORY_F,Rank,WORD_LOWERCASE,ONYM,RSL WORD ID
0,бежевый,Существительное,Цвета,0,1,бежевый,бежевый цвет,0
1,бежевый,Существительное,Цвета,0,1,бежевый,бежевый,1
2,бежевый,Существительное,Цвета,0,1,бежевый,плавый,2
3,белокурый,Прилагательное,Цвета,0,2,белокурый,светловолосый,3
4,белокурый,Прилагательное,Цвета,0,2,белокурый,белобрысый,4


#Translation

In [ ]:
class sentence_translation:
    """docstring"""
 
    def __init__(self, sentence, sentence_id, ner_model, morph_model,RSL_dict):
        """Constructor"""
        self.sentence = sentence  
        self.sentence_id = sentence_id  
        self.ner = ner_model
        self.morph = morph_model
        self.RSL_dict_df = RSL_dict




    '''Get words dataframe'''
    def get_words_dataframe(self):
        tokens = []
        tid = 0

        for _,token in enumerate(tokenize(self.sentence)):
            if token.text != " " and token.text != "" \
                and token.text.strip() not in punctuation:
                tokens.append([self.sentence_id,tid,re.sub('[^0-9А-Яа-яёЁa-zA-Z.!?;, ]', ' ', token.text),token.start,token.stop])
                tid += 1

        self.word_df = pd.DataFrame(tokens,columns= ['SENT_ID', 'WORD_ID', 'WORD','WORD_START', 	'WORD_STOP'])




    ''' Add words lemma'''
    def get_words_lemma(self):
        lemma = pymorphy2.MorphAnalyzer()
        self.word_df['LEMMA'] = self.word_df['WORD'].apply(lambda x: lemma.parse(x)[0].normal_form)




    ''' Add words NER'''
    def get_NER(self):
        #get NER markup
        ner_lst = []

        markup = self.ner(self.sentence)
        for span in markup.spans:
            ner_lst.append([self.sentence_id,span.type,span.start,span.stop])

        ner_df = pd.DataFrame(ner_lst, columns=['SENT_ID', 'NER_TYPE', 'NER_START', 'NER_STOP'])

        #Add NER markup to df
        shp =self.word_df.shape[0]

        self.word_df = self.word_df.merge(ner_df,on =['SENT_ID'], how='left')

        cond = (self.word_df['NER_START']<=self.word_df['WORD_START'])  & (self.word_df['WORD_STOP']<=self.word_df['NER_STOP'])                                                                                                        
        self.word_df.loc[~cond,['NER_START','NER_STOP','NER_TYPE']] = np.nan
        self.word_df = self.word_df.sort_values(['WORD_ID', 'NER_TYPE']).drop_duplicates(['WORD_ID'], keep='first')

        assert shp == self.word_df.shape[0]




    ''' Part of speech tagging'''
    def get_part_of_speech(self):
        #get Part of speech markup
        morph_lst = []
        chunk = [list(self.word_df['WORD'])]

        for sid, s in enumerate(chunk):
            markup = self.morph(s)
            for wid,token in enumerate(markup.tokens):
                morph_lst.append([self.sentence_id, wid, token.text,token.tag])

        morph_df = pd.DataFrame(morph_lst, columns=['SENT_ID', 'WORD_ID', 'WORD', 'MORPH_TAG'])

        #Add Part of speech markup to df
        shp =self.word_df.shape[0]


        self.word_df = self.word_df.merge(morph_df,on =['SENT_ID','WORD_ID','WORD'], how='left')

        self.word_df['PART OF SPEECH ENG'] = [m[:m.find('|')] for m in self.word_df['MORPH_TAG'] ]

        assert shp == self.word_df.shape[0]




    ''' Find collocations in dict'''
    def get_collocations(self):
        #get collocations markup
        colloc_list = [[self.sentence_id,o,self.sentence.find(str(o)),self.sentence.find(str(o))+len(o)]\
                          for o in set(self.RSL_dict_df['ONYM']) \
                          if (self.sentence.find(str(o))>0) & (len(str(o).split())>1)]

        colloc_df = pd.DataFrame(colloc_list, columns=['SENT_ID', 'COLLOCATION', 'COL_START', 'COL_STOP'])
        
        #Add collocations markup to df
        shp = self.word_df.shape[0]

        self.word_df = self.word_df.merge(colloc_df,on =['SENT_ID'], how='left')

        cond = (self.word_df['COL_START']<=self.word_df['WORD_START'])  & (self.word_df['WORD_STOP']<=self.word_df['COL_STOP'])
                                                                                                        
        self.word_df.loc[~cond,['COLLOCATION', 'COL_START', 'COL_STOP']] = np.nan
        self.word_df = self.word_df.sort_values(['WORD_ID', 'COLLOCATION']).drop_duplicates(['WORD_ID'], keep='first')

        assert shp == self.word_df.shape[0]




    ''' Find words in dict'''
    def find_words_in_dict(self):
        #Add ONYMs markup to df
        self.word_df = self.word_df.merge(self.RSL_dict_df[self.RSL_dict_df['ONYM'].isin(self.word_df['LEMMA'])]\
                                          .groupby(['ONYM'])['WORD_LOWERCASE'].count(),left_on='LEMMA',\
                                                    right_on='ONYM', how = 'left', suffixes=['','ONYM'])
        
        #Add WORDs markup to df
        self.word_df = self.word_df.merge(self.RSL_dict_df[self.RSL_dict_df['WORD'].isin(self.word_df['LEMMA'])]\
                                          .drop_duplicates(['WORD', 	'PART_OF_SPEECH', 	'TOPIC_CATEGORY' ])\
                                          .groupby(['WORD'])['WORD_LOWERCASE'].count(),left_on='LEMMA',\
                                                    right_on='WORD', how = 'left', suffixes=['','WORD'])
        
        self.word_df.columns = list(self.word_df.columns[:-2]) + ['CNT ONYM IN DICT','CNT WORD IN DICT']

        self.word_df['CNT WORD IN DICT'] = np.where(self.word_df['CNT WORD IN DICT'].isnull(),0, self.word_df['CNT WORD IN DICT'])
        self.word_df['CNT ONYM IN DICT'] = np.where(self.word_df['CNT ONYM IN DICT'].isnull(),0, self.word_df['CNT ONYM IN DICT'])




    ''' Word sentence disambigulation (WSD)'''
    def disambiguate(self,word_, context_, word_type ):
        # find candidates 
        subset = self.RSL_dict_df[self.RSL_dict_df[word_type].isin([word_])]
        subset = subset.drop_duplicates([word_type, 'WORD_LOWERCASE', 	'PART_OF_SPEECH', 	'TOPIC_CATEGORY'])
        candidates_context = list(subset[['WORD_LOWERCASE','PART_OF_SPEECH','TOPIC_CATEGORY']].apply(lambda x: ' '.join(x), axis = 1))
        candidates = subset['WORD_LOWERCASE']
        candidates_id = subset['RSL WORD ID']
        collocation_f_list = [len(c.split())>1 for c in candidates]

        #find vector for sentence
        _,_,_,_,_,text_vector,_ = bert([' '.join(t.text for t in tokenize(context_) if t.text != word_)])

        # print(candidates_context)
        #find vector for words in RSL dict
        candidate_vectors = []
        for id, c in enumerate(candidates):
            ##if collocation, find  own vector
            # if True:
            if (collocation_f_list[id]) | (word_type == 'ONYM'):
    
                _,_,_,_,_,word_vector,_ = bert(c)
                candidate_vectors.append(word_vector)
            ##if single word, find context vector
            else:
                word_tokens,word_vectors,_,_,_,_,_ = bert([candidates_context[id]])
                idx = list(word_tokens[0]).index(c)
                word_vector = word_vectors[0][idx]
                candidate_vectors.append(word_vector)
            
        if (collocation_f_list[id]) | (word_type == 'ONYM'):
            scores = [np.dot(text_vector, v.T)[0] for v in candidate_vectors[0]]
        else:
            scores = [np.dot(text_vector, v.T)[0] for v in candidate_vectors]
        return dict(zip(candidates_id, scores))




    ''' WSD markup'''
    def get_WSD_markup(self):
        cond_word = (self.word_df['CNT WORD IN DICT']>1) & (self.word_df['NER_TYPE'].isnull())\
                      & (~self.word_df['PART OF SPEECH ENG'].isin(['NUM','NU'])) 
        cond_onym = (self.word_df['CNT WORD IN DICT']==0) & (self.word_df['CNT ONYM IN DICT']>1)\
                      & (self.word_df['NER_TYPE'].isnull()) & (~self.word_df['PART OF SPEECH ENG'].isin(['NUM','NU'])) 

        self.word_df['FINAL WORD FOR VISUALISATION'] = ''
        self.word_df['FINAL WORD FOR RSL ID'] = 0
        self.word_df['FINAL WORD FOR DESCRIPION'] = 'DACTIL'
        self.word_df['SIGN'] = 0
        self.word_df['TYPE'] = ''

        #disambigulate Words
        for ww in self.word_df[cond_word]['LEMMA']:
            rank_list = self.disambiguate(word_ = ww, context_ = ' '.join(self.word_df['LEMMA']),word_type = 'WORD')
            best_word = self.RSL_dict_df[self.RSL_dict_df['RSL WORD ID'] == max(rank_list, key=rank_list.get)]

            self.word_df.loc[self.word_df['LEMMA'] == ww,'FINAL WORD FOR VISUALISATION'] = best_word['WORD'].values[0]
            self.word_df.loc[self.word_df['LEMMA'] == ww,'FINAL WORD FOR RSL ID'] = best_word['RSL WORD ID'].values[0]
            self.word_df.loc[self.word_df['LEMMA'] == ww,'FINAL WORD FOR DESCRIPION'] = 'DICT'
            self.word_df.loc[self.word_df['LEMMA'] == ww,'SIGN'] = 1
            self.word_df.loc[self.word_df['LEMMA'] == ww,'TYPE'] = 'WORD SIGN WDS'

        #disambigulate Onyms
        for ww in self.word_df[cond_onym]['LEMMA']:
            rank_list = self.disambiguate(word_ = ww, context_ = ' '.join(self.word_df['LEMMA']),word_type = 'ONYM')
            best_word = self.RSL_dict_df[self.RSL_dict_df['RSL WORD ID'] == max(rank_list, key=rank_list.get)]

            self.word_df.loc[self.word_df['LEMMA'] == ww,'FINAL WORD FOR VISUALISATION'] = best_word['ONYM'].values[0]
            self.word_df.loc[self.word_df['LEMMA'] == ww,'FINAL WORD FOR RSL ID'] = best_word['RSL WORD ID'].values[0]
            self.word_df.loc[self.word_df['LEMMA'] == ww,'FINAL WORD FOR DESCRIPION'] = 'ONYM'
            self.word_df.loc[self.word_df['LEMMA'] == ww,'SIGN'] = 1
            self.word_df.loc[self.word_df['LEMMA'] == ww,'TYPE'] = 'ONYM SIGN WDS'




    ''' Get final markup'''
    def final_markup(self):
        self.get_words_dataframe()
        self.get_words_lemma()
        self.get_NER()
        self.get_part_of_speech()
        self.get_collocations()
        self.find_words_in_dict()
        self.get_WSD_markup()

        cond_final = (~self.word_df['COLLOCATION'].isnull()) & (self.word_df['FINAL WORD FOR VISUALISATION'] == '')
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['LEMMA'] ,self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['FINAL WORD FOR DESCRIPION'] = np.where(cond_final, 'DICT', self.word_df['FINAL WORD FOR DESCRIPION'])
        self.word_df['SIGN']  = np.where(cond_final, 1, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'COLLOCATION DICT SIGN', self.word_df['TYPE'])

        cond_final = (~self.word_df['NER_TYPE'].isnull()) & (self.word_df['FINAL WORD FOR VISUALISATION'] == '') & (self.word_df['CNT WORD IN DICT']>0)
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['LEMMA'], self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['FINAL WORD FOR DESCRIPION'] = np.where(cond_final,'DICT',self.word_df['FINAL WORD FOR DESCRIPION'])
        self.word_df['SIGN']  = np.where(cond_final, 1, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'NER SIGN', self.word_df['TYPE'])

        cond_final = (~self.word_df['NER_TYPE'].isnull()) & (self.word_df['FINAL WORD FOR VISUALISATION'] == '') & (self.word_df['CNT WORD IN DICT'] == 0)
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['WORD'], self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['SIGN']  = np.where(cond_final, 0, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'NER DACTIL', self.word_df['TYPE'])

        cond_final = (self.word_df['PART OF SPEECH ENG'].isin(['NUM','NU'])) & (self.word_df['FINAL WORD FOR VISUALISATION'] == '')
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['LEMMA'] , self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['FINAL WORD FOR DESCRIPION'] = np.where(cond_final,'DICT',self.word_df['FINAL WORD FOR DESCRIPION'])
        self.word_df['SIGN']  = np.where(cond_final, 1, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'NUMB SIGN', self.word_df['TYPE'])

        cond_final = (self.word_df['CNT WORD IN DICT']>0) & (self.word_df['FINAL WORD FOR VISUALISATION'] == '')
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['LEMMA'] ,self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['FINAL WORD FOR DESCRIPION'] = np.where(cond_final, 'DICT', self.word_df['FINAL WORD FOR DESCRIPION'])
        self.word_df['SIGN']  = np.where(cond_final, 1, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'DICT SIGN', self.word_df['TYPE'])

        cond_final = (self.word_df['CNT ONYM IN DICT']==1) & (self.word_df['FINAL WORD FOR VISUALISATION'] == '')
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['LEMMA'], self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['FINAL WORD FOR DESCRIPION'] = np.where(cond_final, 'ONYM',self.word_df['FINAL WORD FOR DESCRIPION'])
        self.word_df['SIGN']  = np.where(cond_final, 1, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'ONYM DICT SIGN', self.word_df['TYPE'])

        cond_final = self.word_df['FINAL WORD FOR VISUALISATION'] == ''
        self.word_df['FINAL WORD FOR VISUALISATION'] = np.where(cond_final, self.word_df['WORD'], self.word_df['FINAL WORD FOR VISUALISATION'])
        self.word_df['SIGN']  = np.where(cond_final, 0, self.word_df['SIGN'])
        self.word_df['TYPE'] = np.where(cond_final, 'NOT DICT SIGN', self.word_df['TYPE'])


## Get sentences dataframe

In [ ]:
def translate_text(text_,ner_, morph_, RSL_dict_):
    sent_list_ = []

    for id_,sent_ in enumerate(sentenize(text_)):
        #get sentence
        sent_list_.append([id_, sent_.text])

    #sentences df
    sent_df_ = pd.DataFrame(sent_list_,columns= ['ID', 'SENTENCE'])
    sent_df_.head()

    #translation
    translation_df_ = pd.DataFrame()
    for _,row_ in tqdm(sent_df_.iterrows()):
        sentence_ = row_['SENTENCE']
        sentence_id_ = row_['ID']

        translation_ = sentence_translation(sentence = sentence_,
                                          sentence_id = sentence_id_,
                                          ner_model = ner_,
                                          morph_model = morph_,
                                          RSL_dict = RSL_dict_)
        translation_.final_markup()

        translation_df_ = pd.concat([translation_df_, translation_.word_df], ignore_index=True)

    return translation_df_

In [ ]:
def pivot_tbl(tr_df, ind_lst, val_lst):
    pvt = tr_df.pivot_table(index=ind_lst,
                                 values = val_lst,
                                 aggfunc='count')/tr_df.shape[0]
    return (pvt.reset_index().style
    .format({val_lst[0]: lambda x: round(x,4),
            })
    .bar(subset=[val_lst[0]],
          align='mid',
          color=['coral', 'lightblue'],
          vmin=pvt[val_lst[0]].min(),
          vmax=pvt['WORD_ID'].max()
        )
    .set_properties(**{'text-align': 'center'}, subset=val_lst[0])
    )

## My text

In [ ]:
translation_df = translate_text(text_= text,ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

145it [07:19,  3.03s/it]


In [ ]:
pivot_tbl(tr_df = translation_df, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0075
1,DACTIL,NOT DICT SIGN,0.1341
2,DICT,COLLOCATION DICT SIGN,0.0027
3,DICT,DICT SIGN,0.4242
4,DICT,NER SIGN,0.0005
5,DICT,NUMB SIGN,0.0208
6,DICT,WORD SIGN WDS,0.1815
7,ONYM,ONYM DICT SIGN,0.0825
8,ONYM,ONYM SIGN WDS,0.1464


In [ ]:
translation_df.to_csv(path + 'text translation_df.csv')

## RuAdupt
[RuAdupt](https://github.com/Digital-Pushkin-Lab/RuAdapt)

In [ ]:
#open file
with open("/content/drive/MyDrive/Diplom/Generation/s2t/summary.txt", encoding= 'utf-8') as txt:
  text_simple_bleu = txt.read()
text_simple_bleu

'Оставшись один на перекрестке, князь осмотрелся кругом, быстро перешел через улицу, близко подошел к освещенному окну одной дачи. Я решилась говорить с вами об одном деле. И объясните мне леди Эстер? Почему именно ей завещано? Какая связь? Вот что выяснить Вы купили? Хамите. На другой день, в 11 часов утра, Вронский выехал на станцию Петербургской железной дороги. Он не ошибся. Не успели еще замолкнуть раскаты и переливы председательского голоса. Они несли. Один держал его за руки. С тем, кто не испугался бы. Что-нибудь? Закажи порцию селянки Я подумаю Узнаю! - воскликнул Вася. Узнаю Колю Шипучин. Позвольте Берет от нее прошение. Боцмана засвистали. Все матросы были на палубе. У вас очень много шипящих букв. У нас язык легкий. А у вас свистят. Вершинин. О чем? Давайте помечтаем о той жизни, какая будет после нас. Балаганов, - Шура Балаганов. Но и в Западном отряде готовились основательно. Штурм был назначен на сегодня, и все говорили. Эраст Петрович начертил для Вари прутиком все туре

In [ ]:
nn = 250

subsample = np.random.choice(df_bleu.index, nn, replace=False)
bleu_text = df_bleu.loc[subsample].apply(lambda x: ' '.join(x),axis = 0)

len([s for s in sentenize(bleu_text[0])])

1073

In [ ]:
bleu_text = pd.read_csv(path + 'sample.csv')['0']
bleu_text

0    Оставшись один на перекрестке, князь осмотрелс...
1    Аглая ушла, и князь прочитал: «Завтра в семь ч...
2    b1 b1 b1_c1 b1_c1 b1 b1_c1 b1 b1_c1 b1_c1 a2_b...
Name: 0, dtype: object

In [ ]:
translation_df_bleu_full = translate_text(text_= bleu_text[0],ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

166it [11:23,  4.12s/it]


In [ ]:
translation_df_bleu_short = translate_text(text_= bleu_text[1],ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

159it [09:35,  3.62s/it]


In [ ]:
translation_df_bleu_simple = translate_text(text_= text_simple_bleu,ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

120it [03:05,  1.55s/it]


In [ ]:
# translation_df_bleu_full.to_csv(path + 'text translation_df_bleu_full.csv')
# translation_df_bleu_short.to_csv(path + 'text translation_df_bleu_short.csv')

In [ ]:
pivot_tbl(tr_df = translation_df_bleu_full, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0556
1,DACTIL,NOT DICT SIGN,0.188
2,DICT,DICT SIGN,0.4076
3,DICT,NER SIGN,0.0058
4,DICT,NUMB SIGN,0.0129
5,DICT,WORD SIGN WDS,0.084
6,ONYM,ONYM DICT SIGN,0.064
7,ONYM,ONYM SIGN WDS,0.1822


In [ ]:
pivot_tbl(tr_df = translation_df_bleu_short, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0668
1,DACTIL,NOT DICT SIGN,0.1888
2,DICT,COLLOCATION DICT SIGN,0.0008
3,DICT,DICT SIGN,0.4134
4,DICT,NER SIGN,0.0031
5,DICT,NUMB SIGN,0.0155
6,DICT,WORD SIGN WDS,0.0894
7,ONYM,ONYM DICT SIGN,0.0505
8,ONYM,ONYM SIGN WDS,0.1717


In [ ]:
pivot_tbl(tr_df = translation_df_bleu_simple, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0803
1,DACTIL,NOT DICT SIGN,0.1285
2,DICT,DICT SIGN,0.4257
3,DICT,NER SIGN,0.0147
4,DICT,NUMB SIGN,0.0094
5,DICT,WORD SIGN WDS,0.0817
6,ONYM,ONYM DICT SIGN,0.0643
7,ONYM,ONYM SIGN WDS,0.1954


## RuSimpleSentEval
[RuSimpleSentEval](https://github.com/dialogue-evaluation/RuSimpleSentEval/blob/main/dev_sents.csv)

In [ ]:
#open file
with open("/content/drive/MyDrive/Diplom/Generation/s2t/summary rsse.txt", encoding= 'utf-8') as txt:
  text_simple_rsse = txt.read()
text_simple_rsse

"14 декабря 1944 года рабочий посёлок Ички был переименован. 1960 году была выпущена модель 172A. Изменения: хвостовое оперение. 1960 году была выпущена модель 172A. Изменения: хвостовое оперение. 1960 году была выпущена модель 172A. Изменения: хвостовое оперение. 1960 году была выпущена модель 172A. Изменения: хвостовое оперение. 26 августа 2014 года Болдок принял участие в памятном матче Кубка лиги. 26 августа 2014 года Болдок принял участие в памятном матче Кубка лиги. Armory - клиент с различными функциями для повышения безопасности. Concert for Diana (рус. Концерт для Дианы). Concert for Diana (рус. Концерт для Дианы). Concert for Diana (рус. Концерт для Дианы). PlayStation 4 не предусматривает региональной блокировки игр, так что игру можно запускать на консоли из другого региона. PlayStation 4 не предусматривает региональной блокировки игр, так что игру можно запускать на консоли из другого региона. Python популярен среди индивидуальных разработчиков. Python популярен среди инди

In [ ]:
df_rsse = pd.read_csv('https://raw.githubusercontent.com/dialogue-evaluation/RuSimpleSentEval/main/dev_sents.csv', sep= ',',index_col=0, error_bad_lines=False)
df_rsse.head()

,INPUT:source,OUTPUT:output
3,14 декабря 1944 года рабочий посёлок Ички был ...,14 декабря 1944 года рабочий посёлок Ички пере...
4,1960 году была выпущена модель 172A. Изменени...,В 1960 году вышла модель 172А. Отличие в хвост...
5,1960 году была выпущена модель 172A. Изменени...,В выпущенной в 1960 году модель имела изменени...
6,1960 году была выпущена модель 172A. Изменени...,Изменения: в хвосте и руле направления с обрат...
7,1960 году была выпущена модель 172A. Изменени...,"Модель 172А с другим хвостовым оперением, кре..."


In [ ]:
rsse_text = df_rsse.apply(lambda x: ' '.join(x),axis = 0)

len([s for s in sentenize(rsse_text[0])])

3397

In [ ]:
translation_df_rsse_full = translate_text(text_= rsse_text[0],ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

In [ ]:
# translation_df_rsse_full.to_csv(path + 'translation_df_rsse_full.csv')

In [ ]:
translation_df_rsse_short = translate_text(text_= rsse_text[1],ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

2869it [3:21:21,  4.21s/it]


In [ ]:
translation_df_rsse_short.to_csv(path + 'translation_df_rsse_short.csv')

In [ ]:
translation_df_rsse_simple = translate_text(text_= text_simple_rsse,ner_ = ner, morph_ = morph, RSL_dict_ = RSL_dict)

3360it [3:43:22,  3.99s/it]


In [ ]:
translation_df_rsse_simple.to_csv(path + 'translation_df_rsse_short.csv')

In [ ]:
pivot_tbl(tr_df = translation_df_rsse_full, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0529
1,DACTIL,NOT DICT SIGN,0.1528
2,DICT,COLLOCATION DICT SIGN,0.0022
3,DICT,DICT SIGN,0.3702
4,DICT,NER SIGN,0.0179
5,DICT,NUMB SIGN,0.0108
6,DICT,WORD SIGN WDS,0.172
7,ONYM,ONYM DICT SIGN,0.051
8,ONYM,ONYM SIGN WDS,0.1702


In [ ]:
pivot_tbl(tr_df = translation_df_rsse_short, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0632
1,DACTIL,NOT DICT SIGN,0.1324
2,DICT,COLLOCATION DICT SIGN,0.0015
3,DICT,DICT SIGN,0.3704
4,DICT,NER SIGN,0.0208
5,DICT,NUMB SIGN,0.01
6,DICT,WORD SIGN WDS,0.1789
7,ONYM,ONYM DICT SIGN,0.0468
8,ONYM,ONYM SIGN WDS,0.176


In [ ]:
pivot_tbl(tr_df = translation_df_rsse_simple, ind_lst = ['FINAL WORD FOR DESCRIPION','TYPE'], val_lst = ['WORD_ID'])

,FINAL WORD FOR DESCRIPION,TYPE,WORD_ID
0,DACTIL,NER DACTIL,0.0654
1,DACTIL,NOT DICT SIGN,0.1406
2,DICT,COLLOCATION DICT SIGN,0.0019
3,DICT,DICT SIGN,0.3572
4,DICT,NER SIGN,0.0187
5,DICT,NUMB SIGN,0.0126
6,DICT,WORD SIGN WDS,0.1676
7,ONYM,ONYM DICT SIGN,0.0524
8,ONYM,ONYM SIGN WDS,0.1836


# Metrics

### Word Mover's Distance

In [ ]:
def count_WMD(df_, org_nm, tr_nm, group_by = 'SENT_ID'):
    sent_org = df_.groupby([group_by])[org_nm].apply(lambda x: ' '.join(x))
    sent_tr = df_.groupby([group_by])[tr_nm].apply(lambda x: ' '.join(x))

    distance_lst = []
    for si, s in enumerate(sent_org):
        distance = model.wmdistance(s, sent_tr[si])
        distance_lst.append(distance)

    return distance_lst

def count_WMD2( sent_org, sent_tr):

    distance_lst = []
    for si, s in enumerate(sent_org):
        distance = model.wmdistance(s, sent_tr[si])
        distance_lst.append(distance)

    return distance_lst

## My text

In [ ]:
dist_tr = count_WMD(df_ = translation_df, org_nm = 'WORD', tr_nm = 'FINAL WORD FOR VISUALISATION', group_by = 'SENT_ID')
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  0.5508629913226548 
 median:  0.5080463896017212


## RuAdupt
[RuAdupt](https://github.com/Digital-Pushkin-Lab/RuAdapt)

In [ ]:
dist_tr = count_WMD(df_ = translation_df_bleu_short, org_nm = 'WORD', tr_nm = 'FINAL WORD FOR VISUALISATION', group_by = 'SENT_ID')
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  0.7333030001316564 
 median:  0.6458481574074596


In [ ]:
dist_tr = count_WMD(df_ = translation_df_bleu_full, org_nm = 'WORD', tr_nm = 'FINAL WORD FOR VISUALISATION', group_by = 'SENT_ID')
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  0.6301232709815369 
 median:  0.5617961103826231


In [ ]:
t1 = list(translation_df_bleu_simple.groupby(['SENT_ID'])['FINAL WORD FOR VISUALISATION'].apply(lambda x: ' '.join(x)))

In [ ]:
t2 = list(translation_df_bleu_short.groupby(['SENT_ID'])['WORD'].apply(lambda x: ' '.join(x)))

In [ ]:

dist_tr = count_WMD2(t1,t2)
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  2.534064166205053 
 median:  2.477215493881001


## RuSimpleSentEval
[RuSimpleSentEval](https://github.com/dialogue-evaluation/RuSimpleSentEval/blob/main/dev_sents.csv)

In [ ]:
dist_tr = count_WMD(df_ = translation_df_rsse_short, org_nm = 'WORD', tr_nm = 'FINAL WORD FOR VISUALISATION', group_by = 'SENT_ID')
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  0.43708613296541204 
 median:  0.4091530925634533


In [ ]:
dist_tr = count_WMD(df_ = translation_df_rsse_full, org_nm = 'WORD', tr_nm = 'FINAL WORD FOR VISUALISATION', group_by = 'SENT_ID')
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  0.38324368181691704 
 median:  0.36823796575190004


In [ ]:
t1 = list(translation_df_rsse_simple.groupby(['SENT_ID'])['FINAL WORD FOR VISUALISATION'].apply(lambda x: ' '.join(x)))

In [ ]:
t2 = list(translation_df_rsse_simple.groupby(['SENT_ID'])['WORD'].apply(lambda x: ' '.join(x)))

In [ ]:

dist_tr = count_WMD2(t1,t2)
print('mean: ', np.mean(dist_tr),'\n',
'median: ',np.median(dist_tr))

mean:  0.4548210974365499 
 median:  0.4327186583854721


In [ ]:
# mi = distance_lst.index(max(distance_lst))
# distance = model.wmdistance(sent_org[mi], sent_tr[mi])
# print ('distance between s2 and s3 = %.4f' % distance
#        , '\n'
#        ,sent_org[mi] , '\n', sent_tr[mi])

distance between s2 and s3 = 1.6040 
 Что такое с вами 
 что такой с вы
